# Indledende analyse af data

Denne notebook indeholder en *absolut minimal* analyse af de data, vi vil anvende til at udvikle vores skade-ML-model. 

Analysens formål er at få et overordnet indtryk af data.

Notebook'en kan forhåbentlig give inspiration til nyttige greb, der kan foretages i forbindelse med analyse af data i Python.

## Import af python-afhængigheder

Først importerer vi de python-moduler og -pakker, som vi betinger på i analysen.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import ShuffleSplit

## Indlæsning af skade-data

Vi indlæser først de "rå" skades-data. 

**Bemærk**, data for 2019 er forlods udtaget som *test*-sæt, som vi skal bruge til - når modeludviklingen engang er afsluttet - at få et unbiased estimat på den endelige models prædiktive performance på nye, usete datapunkter. 

Det skal spejle brugssituationen, hvor modellen skal anvendes på nye kunder for et nyt år.

In [ ]:
# indlæs data for 2015-18
df = pd.read_parquet("data/train.parquet")

# dan binomial "target"-variabel, der angiver, om der har været skade(r) eller ej
y = df["skadesantal"].values > 0

print(
    "Antal observationer opdelt på år:\n",
    df.groupby("aar").size().reset_index(name="count"),
)
print("Antal observationer med/uden skade(r): ", sum(y), "/", len(y) - sum(y))

Det bemærkes, at der er relativt få "sande" observationer i datasættet. Derfor siges datasættet at være [ubalanceret](https://developers.google.com/machine-learning/data-prep/construct/sampling-splitting/imbalanced-data), hvilket man skal være opmærksom på i modeludviklingen.

## Data splitting

Observationerne fra 2015-2018 splittes tilfældigt i to datasæt til hhv. træning (80%) og validering (20%).

Bemærk, vi instantierer *ShuffleSplit*-klassen. Dette er et eksempel på [objekt-orienteret-programmering](https://en.wikipedia.org/wiki/Object-oriented_programming), som anvendes i Python i stor stil (og som vi vil anvende i stor stil her på workshoppen). 

In [ ]:
# instantierer generator til split af data, random state sættes for at kunne reproducere split
split_generator = ShuffleSplit(n_splits=1, train_size=0.8, random_state=42)

# generér indices for de to splits
train_index, val_index = next(split_generator.split(df))

# opdel data i trænings- og valideringssæt
X_train, y_train = df.iloc[train_index], y[train_index]
X_val, y_val = df.iloc[val_index], y[val_index]

print(
    "Antal observationer i trænings- og valideringssæt: ", len(y_train), "/", len(y_val)
)

Den resterende del af analysen udføres alene på trænings-datasættet.

## Opsummering af data

Et kort oprids af data kan fås med metoden [`info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) for `pandas.DataFrame`. 

En anden nyttig metode er [`describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html).

In [ ]:
X_train.info()

Variablene ses at være en blanding af numeriske og kategoriske. 

En række variable har missing-værdier, hvilket skal håndteres i forbindelse med udviklingen af vores ML-modeller.

### Plots

For at få indtryk af fordelingerne af de enkelte variable, kan histogrammer for de numeriske variable i datasættet plottes uden videre med [`hist()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html).

In [ ]:
X_train.hist(layout=(5, 3), figsize=(15, 10), bins=50)
plt.show()

Fordelingerne af de kategoriske variable i datasættet visualiseres med box plots.

In [ ]:
for col in X_train.select_dtypes(include=["category"]).columns:
    X_train[col].value_counts().plot(kind="bar")
    plt.title(col)
    plt.show()

Er der noget, I hæfter jer ved? Opmærksomhedspunkter?